In [0]:
from pyspark.sql.functions import when
df = df.withColumn("label", when(df.income == "<=50K", 0).otherwise(1))

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-6882951664267114>, line 2
      1 from pyspark.sql.functions import when
----> 2 df = df.withColumn("label", when(df.income == "<=50K", 0).otherwise(1))

NameError: name 'df' is not defined

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

assembler_lr = VectorAssembler(
    inputCols=["age","education_num","capital_gain","capital_loss"],
    outputCol="features"
)

lr_data = assembler_lr.transform(df) \
    .select("features","hours_per_week") \
    .withColumnRenamed("hours_per_week","label")

lr_model = LinearRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=20
).fit(lr_data)

print("RMSE:", lr_model.summary.rootMeanSquaredError)
print("R2:", lr_model.summary.r2)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-6882951664267113>, line 9
      2 from pyspark.ml.feature import VectorAssembler
      4 assembler_lr = VectorAssembler(
      5     inputCols=["age","education_num","capital_gain","capital_loss"],
      6     outputCol="features"
      7 )
----> 9 lr_data = assembler_lr.transform(df) \
     10     .select("features","hours_per_week") \
     11     .withColumnRenamed("hours_per_week","label")
     13 lr_model = LinearRegression(
     14     featuresCol="features",
     15     labelCol="label",
     16     maxIter=20
     17 ).fit(lr_data)
     19 print("RMSE:", lr_model.summary.rootMeanSquaredError)

NameError: name 'df' is not defined

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

num_cols = ["age","education_num","capital_gain","capital_loss","hours_per_week"]

assembler = VectorAssembler(inputCols=num_cols, outputCol="features")
data = assembler.transform(df).select("features","label")

lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=20)
model = lr.fit(data)

pred = model.transform(data)
pred.select("label","prediction","probability").show(5, truncate=False)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-6882951664267112>, line 7
      4 num_cols = ["age","education_num","capital_gain","capital_loss","hours_per_week"]
      6 assembler = VectorAssembler(inputCols=num_cols, outputCol="features")
----> 7 data = assembler.transform(df).select("features","label")
      9 lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=20)
     10 model = lr.fit(data)

NameError: name 'df' is not defined

In [0]:
from pyspark.sql.functions import when
df = df.withColumn("label", when(df.income == "<=50K", 0).otherwise(1))

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-6882951664267111>, line 2
      1 from pyspark.sql.functions import when
----> 2 df = df.withColumn("label", when(df.income == "<=50K", 0).otherwise(1))

NameError: name 'df' is not defined

In [0]:
from pyspark.sql.functions import when
df = df.withColumn("label", when(df.income == "<=50K", 0).otherwise(1))

In [0]:
from pyspark.sql.functions import col, count, when
rows = df.count()
cols = len(df.columns)
dtypes = df.dtypes
missing_df = df.select([
    (count(when(col(c).isNull(), c)) / rows * 100).alias(c)
    for c in df.columns
])

print("Number of rows:", rows)
print("Number of columns:", cols)
print("Data types:", dtypes)

print("Missing values percentage per column:")
missing_df.show(truncate=False)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-6882951664267109>, line 2
      1 from pyspark.sql.functions import col, count, when
----> 2 rows = df.count()
      3 cols = len(df.columns)
      4 dtypes = df.dtypes

NameError: name 'df' is not defined

In [0]:
from pyspark.sql.functions import when
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

df_dt = df.withColumn("label", when(df.income == "<=50K", 0).otherwise(1))

assembler = VectorAssembler(
    inputCols=["age","education_num","capital_gain","capital_loss","hours_per_week"],
    outputCol="features"
)

data = assembler.transform(df_dt).select("features","label")
train, test = data.randomSplit([0.8, 0.2], seed=42)

dt = DecisionTreeClassifier(featuresCol="features", labelCol="label", maxDepth=5)
dt_model = dt.fit(train)

pred = dt_model.transform(test)

acc = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy"
).evaluate(pred)

print("Decision Tree Accuracy:", acc)
pred.select("label","prediction","probability").show(10, truncate=False)

---------------------------------------------------------------------------
UnknownException                          Traceback (most recent call last)
File <command-6593347388646493>, line 6
      3 from pyspark.ml.classification import DecisionTreeClassifier
      4 from pyspark.ml.evaluation import MulticlassClassificationEvaluator
----> 6 df_dt = df.withColumn("label", when(df.income == "<=50K", 0).otherwise(1))
      8 assembler = VectorAssembler(
      9     inputCols=["age","education_num","capital_gain","capital_loss","hours_per_week"],
     10     outputCol="features"
     11 )
     13 data = assembler.transform(df_dt).select("features","label")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1736, in DataFrame.__getattr__(self, name)
   1728     raise AttributeError(
   1729         f"'{self.__class__.__name__}' object has no attribute '{name}'. If you are "
   1730         f"attempting to reference a column named '{name}' then please us

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

k_df = df.select("age","education_num","hours_per_week")

assembler_k = VectorAssembler(
    inputCols=["age","education_num","hours_per_week"],
    outputCol="features"
)

k_ready = assembler_k.transform(k_df)

kmeans = KMeans(k=3, seed=42, featuresCol="features")
k_model = kmeans.fit(k_ready)

clusters = k_model.transform(k_ready)
clusters.select("age","education_num","hours_per_week","prediction").show(10)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6593347388646492>, line 14
     11 k_ready = assembler_k.transform(k_df)
     13 kmeans = KMeans(k=3, seed=42, featuresCol="features")
---> 14 k_model = kmeans.fit(k_ready)
     16 clusters = k_model.transform(k_ready)
     17 clusters.select("age","education_num","hours_per_week","prediction").show(10)

File /databricks/python_shell/lib/dbruntime/MLWorkloadsInstrumentation/_pyspark.py:30, in _create_patch_function.<locals>.patched_method(self, *args, **kwargs)
     28 call_succeeded = False
     29 try:
---> 30     result = original_method(self, *args, **kwargs)
     31     call_succeeded = True
     32     return result

File /databricks/python/lib/python3.12/site-packages/pyspark/ml/base.py:203, in Estimator.fit(self, dataset, params)
    201         return self.copy(params)._fit(dataset)
    202     else:
--> 203      

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

assembler_lr = VectorAssembler(
    inputCols=["age","education_num","capital_gain","capital_loss"],
    outputCol="features"
)

lr_data = assembler_lr.transform(df) \
    .select("features","hours_per_week") \
    .withColumnRenamed("hours_per_week","label")

lr_model = LinearRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=20
).fit(lr_data)

print("RMSE:", lr_model.summary.rootMeanSquaredError)
print("R2:", lr_model.summary.r2)

In [0]:
from pyspark.ml.regression import LinearRegression

reg_data = ml_df.select("features", "hours_per_week") \
                .withColumnRenamed("hours_per_week", "label_reg")

lr_model = LinearRegression(featuresCol="features", labelCol="label_reg", maxIter=20).fit(reg_data)

print("RMSE:", lr_model.summary.rootMeanSquaredError)
print("R2:", lr_model.summary.r2)

In [0]:
from pyspark.sql.functions import when
from pyspark.ml.feature import VectorAssembler

df = df.withColumn("label", when(df.income == "<=50K", 0).otherwise(1))

num_cols = ["age","education_num","capital_gain","capital_loss","hours_per_week"]
assembler = VectorAssembler(inputCols=num_cols, outputCol="features")
ml_df = assembler.transform(df).select("features","label", *num_cols)

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

num_cols = ["age","education_num","capital_gain","capital_loss","hours_per_week"]

assembler = VectorAssembler(inputCols=num_cols, outputCol="features")
data = assembler.transform(df).select("features","label")

lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=20)
model = lr.fit(data)

pred = model.transform(data)
pred.select("label","prediction","probability").show(5, truncate=False)

In [0]:
from pyspark.sql.functions import when
df = df.withColumn("label", when(df.income == "<=50K", 0).otherwise(1))

In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

categorical = ["workclass","education","marital_status","occupation",
               "relationship","race","sex","native_country"]

indexers = [StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid="keep")
            for c in categorical]

features = ["age","education_num","capital_gain",
            "capital_loss","hours_per_week"] + [c+"_idx" for c in categorical]

assembler = VectorAssembler(inputCols=features, outputCol="features")

lr = LogisticRegression(featuresCol="features", labelCol="label")

pipeline = Pipeline(stages=indexers + [assembler, lr])

model = pipeline.fit(df)
pred = model.transform(df)

pred.select("income","label","prediction").show(5)

In [0]:
from pyspark.sql.functions import when

df = df.withColumn(
    "label",
    when(df.income == "<=50K", 0).otherwise(1)
)

In [0]:
from pyspark.sql.functions import when

df = df.withColumn(
    "label",
    when(df.income == "<=50K", 0).otherwise(1)
)

In [0]:
columns = [
    "age","workclass","fnlwgt","education","education_num",
    "marital_status","occupation","relationship","race","sex",
    "capital_gain","capital_loss","hours_per_week","native_country","income"
]

df = df.toDF(*columns)
df.show(5)

In [0]:
path = "file:/Workspace/Users/asadnoora339@gmail.com/data/student-mat.csv"

df = spark.read.csv(
    path,
    header=True,
    inferSchema=True,
    sep=";"
)

df.show(5)

---------------------------------------------------------------------------
UnknownException                          Traceback (most recent call last)
File <command-7251727099885932>, line 10
      1 path = "file:/Workspace/Users/asadnoora339@gmail.com/data/student-mat.csv"
      3 df = spark.read.csv(
      4     path,
      5     header=True,
      6     inferSchema=True,
      7     sep=";"
      8 )
---> 10 df.show(5)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1123, in DataFrame.show(self, n, truncate, vertical)
   1122 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1123     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:876, in DataFrame._show_string(self, n, truncate, vertical)
    859     except ValueError:
    860         raise PySparkTypeError(
    861             errorClass="NOT_BOOL",
    862 

In [0]:
path = "/databricks-datasets/adult/adult.data"

df = spark.read.csv(path, header=False, inferSchema=True)

df.show(5)

+---+-----------------+--------+----------+----+-------------------+------------------+--------------+------+-------+------+----+----+--------------+------+
|_c0|              _c1|     _c2|       _c3| _c4|                _c5|               _c6|           _c7|   _c8|    _c9|  _c10|_c11|_c12|          _c13|  _c14|
+---+-----------------+--------+----------+----+-------------------+------------------+--------------+------+-------+------+----+----+--------------+------+
| 39|        State-gov| 77516.0| Bachelors|13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|2174.0| 0.0|40.0| United-States| <=50K|
| 50| Self-emp-not-inc| 83311.0| Bachelors|13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|   0.0| 0.0|13.0| United-States| <=50K|
| 38|          Private|215646.0|   HS-grad| 9.0|           Divorced| Handlers-cleaners| Not-in-family| White|   Male|   0.0| 0.0|40.0| United-States| <=50K|
| 53|          Private|234721.0|      11th| 7.0| Married-c